In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install fastai==2.2.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from typing import Optional
import torch
import transformers
from transformers import AutoModelWithLMHead, PreTrainedTokenizerFast
from fastai.text.all import *
import fastai
import re

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>') 
model = AutoModelWithLMHead.from_pretrained("skt/kogpt2-base-v2")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:973: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
with open('/content/drive/MyDrive/CUAI 2022 하계 컨퍼런스/DATA/서론_텍스트.txt') as f:
   lines = f.read()
len(lines)

410450

In [ ]:
#model input output tokenizer
class TransformersTokenizer(Transform):
   def __init__(self, tokenizer): self.tokenizer = tokenizer
   def encodes(self, x): 
       toks = self.tokenizer.tokenize(x)
       return tensor(self.tokenizer.convert_tokens_to_ids(toks))
   def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

In [ ]:
#split data
train=lines[:int(len(lines)*0.9)]
test=lines[int(len(lines)*0.9):]
splits = [[0],[1]]

In [ ]:
#init dataloader
tls = TfmdLists([train,test], TransformersTokenizer(tokenizer), splits=splits, dl_type=LMDataLoader)

# 튜닝할 필요 있을듯?
batch = 8
seq_len = 256

dls = tls.dataloaders(bs=batch, seq_len=seq_len)

In [ ]:
#gpt2 output is tuple, we need just one val
class DropOutput(Callback):
  def after_pred(self):
    self.learn.pred = self.pred[0]

In [ ]:
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()).to_fp16()
#lr=learn.lr_find()
#print(lr)

In [ ]:
# 이것도 튜닝할 필요 있을듯듯?
learn.fine_tune(2)  # more fine tuning epochs led to way too high perplexity (low perplexity is preferable)

epoch,train_loss,valid_loss,perplexity,time
0,6.040688,6.708315,819.189026,00:19


epoch,train_loss,valid_loss,perplexity,time
0,5.255019,6.330558,561.469727,00:19
1,4.192893,6.155184,471.153656,00:19


In [ ]:
# saving model
learn.export('/content/drive/MyDrive/CUAI 2022 하계 컨퍼런스/Model/test_export.pkl')

# loading the saved model
learn = load_learner('/content/drive/MyDrive/CUAI 2022 하계 컨퍼런스/Model/test_export.pkl')
learn.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [ ]:
prompt="베이지안 통계학은 최고다."
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
preds = learn.model.generate(inp,
                           max_length=156,  # 이것도 튜닝할 필요가 있을듯?
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           repetition_penalty=2.0,  # 이것도 튜닝할 필요가 있을듯?     
                           use_cache=True
                          ) 
tokenizer.decode(preds[0].cpu().numpy())

'베이지안 통계학은 최고다. 그러나 이러한 모형에는 이르지 못한 실정이므로, 본 연구에서는 모의실험을 통하여 분석하고자 한다. 즉, GIS를 이용하여 현장 적용한 실험결과를 토대로 하여 한계상태인자를 추출하는 방법이다. 또한, TSV는 일반적인 수치모형인 CADRM의 그것과 비교 및 분석을 위해 널리 사용되고 있는 것으로 알려져 있다. 따라서 기존의 연구는 주로 인공공법 (CCDKNeomeng), 또는 SNPE/LCA) 등의 다양한 공법의 응용 가능성을 평가하였다. 아울러 여러 가지 문제점들을 복합적으로 고려하기 위하여 실시설계용 프로그램(Increpolarization)를 이용한 최적응력평가 이론을 제시하고, 이를 기반으로 한 결과와 관련된 인력과 기능의 변화를 파악한다. 또한 최근 들어 국내건설공사에 있어서 가장 대표적인'